In [1]:
%cd ..

/Users/floydluo/Library/CloudStorage/GoogleDrive-jjluo@terpmail.umd.edu/.shortcut-targets-by-id/1qNzMmGHCg5Xa63Vw3aKbZdMXvkfT2CgC/MedStar/MS_CODE/FieldNN


# Patient Data Point

In [2]:
from recfldgrn.datapoint import PatientDP, RANGE_SIZE

PID = 'P4'
rec_folder = 'data/ProcData/RecFolder/'
Pat = PatientDP(PID, rec_folder, RANGE_SIZE)
Pat

## Patient Record

In [3]:
RecName = 'P'
df = Pat.get_df_rec(RecName)
df

,PID,age,basicInfo
0,P4,23,Male&A


### Task Selection

In [4]:
TaskName = 'Task2YearXXX'

ELIG_PAT_SET = [f'P{i}' for i in range(0, 8)]
ELIG_PAT_SET

['P0', 'P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7']

In [5]:
import os
import pandas as pd

Tensor_folder = 'data/ProcData/TensorFolder'
TaskTensor_folder = os.path.join(Tensor_folder, TaskName)

if not os.path.exists(TaskTensor_folder): os.makedirs(TaskTensor_folder)

print(TaskTensor_folder)

data/ProcData/TensorFolder/Task2YearXXX


In [6]:
RecName = 'P'

df = pd.DataFrame()
for PID in ELIG_PAT_SET:
    Pat = PatientDP(PID, rec_folder, RANGE_SIZE)
    dfx = Pat.get_df_rec(RecName)
    # make some operation to select good dfx
    dfx = dfx 
    df = pd.concat([df, dfx])
    
print(df.shape)
df

(8, 3)


,PID,age,basicInfo
0,P0,52,Male&B
0,P1,21,Male&B
0,P2,23,Female&B
0,P3,77,Female&B
0,P4,23,Male&A
0,P5,73,Female&D
0,P6,34,Female&A
0,P7,76,Male&C


### Load Meta Info

In [7]:
import os
import pandas as pd

fldgrn_folder = 'data/ProcData/FldGrnInfo'

fullfldgrn_file = os.path.join(fldgrn_folder, RecName + '.p')

df_FieldGrainInfo = pd.read_pickle(fullfldgrn_file)
df_FieldGrainInfo

,RecName,prefix_ids,focal_ids,field_list,field,grain,field2grain,recfield2grain,Vocab
0,P,[],[PID],"[basicInfo, age]",basicInfo,basicInfoDftGrn,basicInfo-basicInfoDftGrn,P@basicInfo-basicInfoDftGrn,"{'idx2v': {0: 'Male', 1: 'B', 2: 'Female', 3: ..."
1,P,[],[PID],"[basicInfo, age]",age,AgeNumeDftGrn,age-AgeNumeDftGrn,P@age-AgeNumeDftGrn,"{'v2idx': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,..."


### RecFldGrn `P@basicInfo-basicInfoDftGrn`

In [8]:
from recfldgrn.grains import Field2Grain_2_get_GrnStrList_Fn
from recfldgrn.grains import Field2Grain_2_get_GrnIdxList_Fn
from recfldgrn.grains import Field2Grain_2_get_GrnWgtList_Fn
from recfldgrn.utils import generate_grain_vocab_info
from recfldgrn.utils import get_compressed_df
from recfldgrn.datapoint import write_df_to_folders

In [9]:
from recfldgrn.utils import get_highorder_input_idx

# def get_highorder_input_idx(df, recfield2grain, 
#                             v2idx, prefix_ids, focal_ids, 
#                             Field2Grain_2_get_GrnIdxList_Fn, 
#                             Field2Grain_2_get_GrnWgtList_Fn):
    
#     RecName, field = recfield2grain.split('-')[0].split('@')
#     grain = recfield2grain.split('-')[-1]
    
#     suffix = '_idx' if 'Nume' not in recfield2grain else '_wgt'
#     if suffix == '_idx':
#         get_grn_idx_fn = Field2Grain_2_get_GrnIdxList_Fn[recfield2grain]
#         df[recfield2grain + suffix] = df[field].apply(lambda x: get_grn_idx_fn(x, v2idx))
#     elif suffix == '_wgt':
#         get_grn_wgt_fn = Field2Grain_2_get_GrnWgtList_Fn[recfield2grain]
#         df[recfield2grain + suffix] = df[field].apply(lambda x: get_grn_wgt_fn(x))
#     else:
#         raise ValueError(f'Incorrect suffix "{suffix}"')
    
    
#     full_recfldgrn_name_c = recfield2grain + suffix
#     top_id = df.columns[0] # need to double check this.
    
#     if len(prefix_ids) > 0:
#         prefix_ids_c = prefix_ids
#         df_Rec = df[prefix_ids_c + [full_recfldgrn_name_c]]
#         for i in range(len(prefix_ids_c)):
#             df_Rec, full_recfldgrn_name_c, prefix_ids_c = get_compressed_df(df_Rec, full_recfldgrn_name_c, prefix_ids_c)
#     else:
#         df_Rec = df[focal_ids + [full_recfldgrn_name_c]]

#     # df_p = df_Rec#.set_index(top_id)
#     df_p = df_Rec.reset_index(drop = True)
#     return df_p

In [10]:
TaskTensor_folder

'data/ProcData/TensorFolder/Task2YearXXX'

In [11]:
recfield2grain = 'P@basicInfo-basicInfoDftGrn'

d = df_FieldGrainInfo[df_FieldGrainInfo['recfield2grain'] == recfield2grain].iloc[0]
v2idx = d['Vocab']['v2idx']
prefix_ids = d['prefix_ids']
focal_ids = d['focal_ids']

df_p = get_highorder_input_idx(df, recfield2grain, 
                               v2idx, prefix_ids, focal_ids,
                               Field2Grain_2_get_GrnIdxList_Fn, 
                               Field2Grain_2_get_GrnWgtList_Fn)



fullTaskTensor_folder = os.path.join(TaskTensor_folder, recfield2grain)
print(df.shape, fullTaskTensor_folder)
if not os.path.exists(fullTaskTensor_folder): os.makedirs(fullTaskTensor_folder)
write_df_to_folders(recfield2grain, fullTaskTensor_folder, df_p)

df_p

(8, 4) data/ProcData/TensorFolder/Task2YearXXX/P@basicInfo-basicInfoDftGrn


,PID,P@basicInfo-basicInfoDftGrn_idx
0,P0,"[0, 1]"
1,P1,"[0, 1]"
2,P2,"[2, 1]"
3,P3,"[2, 1]"
4,P4,"[0, 3]"
5,P5,"[2, 4]"
6,P6,"[2, 3]"
7,P7,"[0, 5]"


### RecFldGrn `P@age-AgeNumeDftGrn`

In [12]:
recfield2grain = 'P@age-AgeNumeDftGrn'


d = df_FieldGrainInfo[df_FieldGrainInfo['recfield2grain'] == recfield2grain].iloc[0]
v2idx = d['Vocab']['v2idx']
prefix_ids = d['prefix_ids']
focal_ids = d['focal_ids']

df_p = get_highorder_input_idx(df, recfield2grain, 
                               v2idx, prefix_ids, focal_ids,
                               Field2Grain_2_get_GrnIdxList_Fn, 
                               Field2Grain_2_get_GrnWgtList_Fn)

df_p

,PID,P@age-AgeNumeDftGrn_wgt
0,P0,"[0, 0, 0, 0, 0, 1, 1.0, 1.0, 1.0, 1.0, 1.0, 1...."
1,P1,"[0, 0, 0, 0, 0, 1, 0.6, 0.0, 0.0, 0.0, 0.0, 0...."
2,P2,"[0, 0, 0, 0, 0, 1, 1.0, 0.0, 0.0, 0.0, 0.0, 0...."
3,P3,"[0, 0, 0, 0, 0, 1, 1.0, 1.0, 1.0, 1.0, 1.0, 1...."
4,P4,"[0, 0, 0, 0, 0, 1, 1.0, 0.0, 0.0, 0.0, 0.0, 0...."
5,P5,"[0, 0, 0, 0, 0, 1, 1.0, 1.0, 1.0, 1.0, 1.0, 1...."
6,P6,"[0, 0, 0, 0, 0, 1, 1.0, 1.0, 1.0, 0.2, 0.0, 0...."
7,P7,"[0, 0, 0, 0, 0, 1, 1.0, 1.0, 1.0, 1.0, 1.0, 1...."


## Encounter Records

In [13]:
RecName = 'EC'
df = Pat.get_df_rec(RecName)
df

,PID,ECID,DT_min,DT_max,BasicInfo
0,P7,EC17,2023-03-18,2023-04-06,A&W


### Task Selection

In [14]:
RecName = 'EC'

df = pd.DataFrame()
for PID in ELIG_PAT_SET:
    Pat = PatientDP(PID, rec_folder, RANGE_SIZE)
    dfx = Pat.get_df_rec(RecName)
    # make some operation to select good dfx
    dfx = dfx 
    df = pd.concat([df, dfx])
    
print(df.shape)
df

(225, 5)


,PID,ECID,DT_min,DT_max,BasicInfo
0,P0,EC18,2023-03-18,2023-04-16,D&X
1,P0,EC140,2023-03-19,2023-04-05,D&X
2,P0,EC23,2023-03-19,2023-04-16,C&W
3,P0,EC141,2023-03-20,2023-04-16,C&Z
4,P0,EC142,2023-03-20,2023-04-15,B&W
...,...,...,...,...,...
3,P6,EC15,2023-03-21,2023-04-09,B&X
4,P6,EC93,2023-03-23,2023-04-04,B&X
5,P6,EC195,2023-03-25,2023-03-25,C&Z
6,P6,EC173,2023-04-05,2023-04-13,C&W


### Load Meta Info

In [15]:
import os
import pandas as pd

fldgrn_folder = 'data/ProcData/FldGrnInfo'

fullfldgrn_file = os.path.join(fldgrn_folder, RecName + '.p')

df_FieldGrainInfo = pd.read_pickle(fullfldgrn_file)
df_FieldGrainInfo

,RecName,prefix_ids,focal_ids,field_list,field,grain,field2grain,recfield2grain,Vocab
0,EC,[PID],[ECID],"[BasicInfo, DT_min]",BasicInfo,BasicDftGrn,BasicInfo-BasicDftGrn,EC@BasicInfo-BasicDftGrn,"{'idx2v': {0: 'X', 1: 'D', 2: 'Y', 3: 'B', 4: ..."
1,EC,[PID],[ECID],"[BasicInfo, DT_min]",DT_min,DTDftGrn,DT_min-DTDftGrn,EC@DT_min-DTDftGrn,"{'idx2v': {0: '_missing', 1: 'Yx', 2: 'Y2010',..."


### RecFldGrn `EC@BasicInfo-BasicDftGrn`

In [16]:
recfield2grain = 'EC@BasicInfo-BasicDftGrn'

d = df_FieldGrainInfo[df_FieldGrainInfo['recfield2grain'] == recfield2grain].iloc[0]
v2idx = d['Vocab']['v2idx']
prefix_ids = d['prefix_ids']
focal_ids = d['focal_ids']

df_p = get_highorder_input_idx(df, recfield2grain, 
                               v2idx, prefix_ids, focal_ids,
                               Field2Grain_2_get_GrnIdxList_Fn, 
                               Field2Grain_2_get_GrnWgtList_Fn)



fullTaskTensor_folder = os.path.join(TaskTensor_folder, recfield2grain)
print(df.shape, fullTaskTensor_folder)
if not os.path.exists(fullTaskTensor_folder): os.makedirs(fullTaskTensor_folder)
write_df_to_folders(recfield2grain, fullTaskTensor_folder, df_p)

df_p

(225, 6) data/ProcData/TensorFolder/Task2YearXXX/EC@BasicInfo-BasicDftGrn


,PID,P-EC@BasicInfo-BasicDftGrn_idx
0,P0,"[[1, 0], [1, 0], [6, 7], [6, 4], [3, 7], [3, 0..."
1,P1,"[[1, 2], [5, 0], [1, 7], [6, 0], [6, 7], [6, 4..."
2,P2,"[[1, 2], [6, 0], [3, 4], [5, 0], [5, 7], [5, 7..."
3,P3,"[[6, 2], [5, 0], [6, 7], [1, 2], [5, 2], [6, 2..."
4,P4,"[[3, 4], [3, 4], [6, 7], [1, 0], [1, 0], [5, 0..."
5,P5,"[[1, 2], [1, 7], [5, 7], [1, 0], [1, 0], [1, 2..."
6,P6,"[[3, 2], [5, 7], [1, 4], [3, 0], [3, 0], [6, 4..."
7,P7,"[[5, 7]]"


### RecFldGrn `EC@DT_min-DTDftGrn`

In [17]:
recfield2grain = 'EC@DT_min-DTDftGrn'

d = df_FieldGrainInfo[df_FieldGrainInfo['recfield2grain'] == recfield2grain].iloc[0]
v2idx = d['Vocab']['v2idx']
prefix_ids = d['prefix_ids']
focal_ids = d['focal_ids']

df_p = get_highorder_input_idx(df, recfield2grain, 
                               v2idx, prefix_ids, focal_ids,
                               Field2Grain_2_get_GrnIdxList_Fn, 
                               Field2Grain_2_get_GrnWgtList_Fn)



fullTaskTensor_folder = os.path.join(TaskTensor_folder, recfield2grain)
print(df.shape, fullTaskTensor_folder)
if not os.path.exists(fullTaskTensor_folder): os.makedirs(fullTaskTensor_folder)
write_df_to_folders(recfield2grain, fullTaskTensor_folder, df_p)

df_p

(225, 7) data/ProcData/TensorFolder/Task2YearXXX/EC@DT_min-DTDftGrn


,PID,P-EC@DT_min-DTDftGrn_idx
0,P0,"[[15, 25, 53, 68, 78, 92, 137], [15, 25, 54, 6..."
1,P1,"[[15, 25, 53, 68, 78, 92, 137], [15, 25, 53, 6..."
2,P2,"[[15, 25, 53, 68, 78, 92, 137], [15, 25, 53, 6..."
3,P3,"[[15, 25, 53, 68, 78, 92, 137], [15, 25, 53, 6..."
4,P4,"[[15, 25, 53, 68, 78, 92, 137], [15, 25, 53, 6..."
5,P5,"[[15, 25, 53, 68, 78, 92, 137], [15, 25, 53, 6..."
6,P6,"[[15, 25, 54, 68, 79, 92, 137], [15, 25, 54, 6..."
7,P7,"[[15, 25, 53, 68, 78, 92, 137]]"


## Diag Record

TODO

In [6]:
# RecName = 'Diag'
# df = Pat.get_df_rec(RecName)
# df

## A1C Record

TODO

In [7]:
# RecName = 'A1C'
# df = Pat.get_df_rec(RecName)
# df

## Smoking Record

TODO

In [8]:
# RecName = 'Smoking'
# df = Pat.get_df_rec(RecName)
# df

## PN Record

TODO

In [9]:
# RecName = 'PN'
# df = Pat.get_df_rec(RecName)
# df

## PNSect Record

TODO

In [10]:
# RecName = 'PNSect'
# df = Pat.get_df_rec(RecName)
# df

## PNSectSent Record

In [18]:
RecName = 'PNSectSent'
df = Pat.get_df_rec(RecName)
df

,PID,ECID,PNID,PNSectID,SentID,Sentence
0,P7,EC17,435,50,397,Particularly who appear top paper stock college.
1,P7,EC17,435,42,334,Chance capital pretty.
2,P7,EC17,435,42,335,Learn force understand reflect important least...
3,P7,EC17,435,42,336,Decade employee too music foot window.
4,P7,EC17,435,42,337,Trade drive fast.
...,...,...,...,...,...,...
83,P7,EC17,435,43,372,Man artist food area character.
84,P7,EC17,435,43,373,Language very back strategy.
85,P7,EC17,435,43,374,Feel budget tough edge read collection however...
86,P7,EC17,435,39,310,Car true daughter guy strategy.


### Task Selection

Based on the tasks, sometime we only use a subset of all records to generate input tensor

In [19]:
RecName = 'PNSectSent'

df = pd.DataFrame()
for PID in ELIG_PAT_SET:
    Pat = PatientDP(PID, rec_folder, RANGE_SIZE)
    dfx = Pat.get_df_rec(RecName)
    # make some operation to select good dfx
    dfx = dfx 
    df = pd.concat([df, dfx])
    
print(df.shape)

(2587, 6)


In [20]:
df

,PID,ECID,PNID,PNSectID,SentID,Sentence
0,P0,EC21,72374,149,1293,Later available sure scientist water out less.
1,P0,EC21,72374,145,1272,Wear relationship down surface section.
2,P0,EC21,72374,145,1271,Concern painting able than.
3,P0,EC21,72374,145,1270,Quite together just fund field.
4,P0,EC21,72374,145,1269,Owner PM building win cultural agent forward.
...,...,...,...,...,...,...
83,P7,EC17,435,43,372,Man artist food area character.
84,P7,EC17,435,43,373,Language very back strategy.
85,P7,EC17,435,43,374,Feel budget tough edge read collection however...
86,P7,EC17,435,39,310,Car true daughter guy strategy.


### Load Meta Info

In [21]:
import os
import pandas as pd

fldgrn_folder = 'data/ProcData/FldGrnInfo'

fullfldgrn_file = os.path.join(fldgrn_folder, RecName + '.p')

df_FieldGrainInfo = pd.read_pickle(fullfldgrn_file)
df_FieldGrainInfo

,RecName,prefix_ids,focal_ids,field_list,field,grain,field2grain,recfield2grain,Vocab
0,PNSectSent,"[PID, ECID, PNID, PNSectID]",[SentID],[Sentence],Sentence,Tk@TknzLLMGrn,Sentence-Tk@TknzLLMGrn,PNSectSent@Sentence-Tk@TknzLLMGrn,{'v2idx': BertTokenizerFast(name_or_path='bert...


### RecFldGrn `PNSectSent@Sentence-Tk@TknzLLMGrn`

In [22]:
recfield2grain = 'PNSectSent@Sentence-Tk@TknzLLMGrn'

from recfldgrn.grains import Field2Grain_2_get_GrnStrList_Fn
from recfldgrn.grains import Field2Grain_2_get_GrnIdxList_Fn
from recfldgrn.grains import Field2Grain_2_get_GrnWgtList_Fn
from recfldgrn.utils import generate_grain_vocab_info
d = df_FieldGrainInfo[df_FieldGrainInfo['recfield2grain'] == recfield2grain].iloc[0]
field = d['field']
grain = d['grain']
Vocab = d['Vocab']
v2idx = Vocab['v2idx']
prefix_ids = d['prefix_ids']

In [23]:
print(v2idx)

BertTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})


In [24]:
suffix = '_idx' if 'Nume' not in recfield2grain else '_wgt'

get_grn_idx_fn = Field2Grain_2_get_GrnIdxList_Fn[recfield2grain]

## Pay attention to this `df`.

df[recfield2grain + suffix] = df[field].apply(lambda x: get_grn_idx_fn(x, v2idx))
df

,PID,ECID,PNID,PNSectID,SentID,Sentence,PNSectSent@Sentence-Tk@TknzLLMGrn_idx
0,P0,EC21,72374,149,1293,Later available sure scientist water out less.,"[101, 2101, 2800, 2469, 7155, 2300, 2041, 2625..."
1,P0,EC21,72374,145,1272,Wear relationship down surface section.,"[101, 4929, 3276, 2091, 3302, 2930, 1012, 102]"
2,P0,EC21,72374,145,1271,Concern painting able than.,"[101, 5142, 4169, 2583, 2084, 1012, 102]"
3,P0,EC21,72374,145,1270,Quite together just fund field.,"[101, 3243, 2362, 2074, 4636, 2492, 1012, 102]"
4,P0,EC21,72374,145,1269,Owner PM building win cultural agent forward.,"[101, 3954, 7610, 2311, 2663, 3451, 4005, 2830..."
...,...,...,...,...,...,...,...
83,P7,EC17,435,43,372,Man artist food area character.,"[101, 2158, 3063, 2833, 2181, 2839, 1012, 102]"
84,P7,EC17,435,43,373,Language very back strategy.,"[101, 2653, 2200, 2067, 5656, 1012, 102]"
85,P7,EC17,435,43,374,Feel budget tough edge read collection however...,"[101, 2514, 5166, 7823, 3341, 3191, 3074, 2174..."
86,P7,EC17,435,39,310,Car true daughter guy strategy.,"[101, 2482, 2995, 2684, 3124, 5656, 1012, 102]"


In [25]:
from recfldgrn.utils import get_compressed_df


prefix_ids_c = prefix_ids
top_id = prefix_ids[0]
full_recfldgrn_name_c = recfield2grain + suffix
df_Rec = df[prefix_ids_c + [full_recfldgrn_name_c]]

for i in range(len(prefix_ids_c)):
    print(prefix_ids_c, full_recfldgrn_name_c)
    df_Rec, full_recfldgrn_name_c, prefix_ids_c = get_compressed_df(df_Rec, 
                                                                    full_recfldgrn_name_c, 
                                                                    prefix_ids_c)
    print(prefix_ids_c, full_recfldgrn_name_c, '\n')
    
df_p = df_Rec.set_index(top_id)
df_p

['PID', 'ECID', 'PNID', 'PNSectID'] PNSectSent@Sentence-Tk@TknzLLMGrn_idx
['PID', 'ECID', 'PNID'] PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx 

['PID', 'ECID', 'PNID'] PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx
['PID', 'ECID'] PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx 

['PID', 'ECID'] PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx
['PID'] EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx 

['PID'] EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx
[] P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx 



,P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx
PID,
P0,"[[[[[101, 2739, 4950, 2025, 1012, 102]], [[101..."
P1,"[[[[[101, 3178, 5987, 1037, 2694, 1012, 102]],..."
P2,"[[[[[101, 2187, 2267, 2312, 3200, 2590, 2369, ..."
P3,"[[[[[101, 2055, 2832, 2574, 2735, 1012, 102]],..."
P4,"[[[[[101, 2085, 2934, 2161, 2283, 1012, 102]],..."
P5,"[[[[[101, 2565, 2653, 2946, 3260, 5293, 3267, ..."
P6,"[[[[[101, 4551, 4299, 3087, 2835, 2017, 2231, ..."
P7,"[[[[[101, 2482, 2995, 2684, 3124, 5656, 1012, ..."
